In [1]:
import numpy as np
import os
import logging
from alertstack.analyse import Analyse
from alertstack.scramble_catalogues.blazar_catalogue import Fermi4FGLBlazarCatalogue, AverageFluxWeightHypothesis,\
    BrightestFluxWeightHypothesis
from alertstack.fixed_catalogues.icecube_neutrino_alerts import CircularisedNeutrinoAlertCatalogue
from pathlib import Path
from alertstack.stats import GammaDistribution

In [2]:
blazar_cache = os.path.join(Path().resolve(), "cache/")

blazar_analysis = Analyse(
    Fermi4FGLBlazarCatalogue(),
    [AverageFluxWeightHypothesis],
    CircularisedNeutrinoAlertCatalogue(),
    cache_dir=blazar_cache,
    clean_cache=True
)

In [3]:
logging.getLogger().setLevel("INFO")

blazar_analysis.iterate_run(
    n_trials=50,
    injection_hypo=AverageFluxWeightHypothesis,
    fraction=0.5,
    nsteps=5,
)

all_res = blazar_analysis.load_results()

sens_threshold = dict()
disc_3_threshold = dict()
disc_5_threshold = dict()

zero_key = 0.0

for key, val in all_res[zero_key].items():
    sens_threshold[key] = np.median(val)
    gd = GammaDistribution(val)
    disc_3_threshold[key] = gd.calculate_discovery_potential(3.)
    disc_5_threshold[key] = gd.calculate_discovery_potential(5.)
    # print(Chi2(val))
    # input("?")

levels = [
    ("Background Median", sens_threshold),
    ("3 Sigma Discovery Potential", disc_3_threshold),
    ("5 Sigma Discovery Potential", disc_5_threshold)
]

for step, res in all_res.items():
    print("Fraction of ASTROPHYISCAL neutrino alerts correlated to source: {0} \n".format(step))

    bkgs = dict()

    for key, val in res.items():
        print(key, np.mean(val), np.median(val), np.std(val))
        val = np.array(val)

        for name, thresh in levels:
            print(thresh[key])
            print("Fraction above {0}: {1}".format(
                name, np.sum(val > thresh[key])/float(len(val))))

100%|██████████| 50/50 [00:17<00:00,  2.86it/s]


Saving to: /Users/robertstein/Code/alertstack/examples/fermi_blazar_neutrino_alert/cache/2021_02_17-13_53_52.pkl
dict_keys([0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5])
Saving to: /Users/robertstein/Code/alertstack/examples/fermi_blazar_neutrino_alert/cache/2021_02_17-13_53_52.pkl
dict_keys([0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5])
      fun: 2390.334002042345
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00109139,  0.00045475, -0.02219167])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 188
      nit: 39
   status: 0
  success: True
        x: array([13.51215986, -3.35207779,  0.73694307])
      fun: 2390.334002042345
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00109139,  0.00045475, -0.02219167])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 188
      nit: 39
   status: 0
  success: True
        x: array([13.51215986, -3.35207779,  0.73694307])
Fraction of ASTR